**코드 조각 1: 프로그램 인자 설정**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/stock/

/content/drive/MyDrive/stock


In [3]:
!pip install import-ipynb

In [4]:
%run /content/drive/MyDrive/stock/utils.ipynb

In [5]:
%run /content/drive/MyDrive/stock/settings.ipynb

In [6]:
%run /content/drive/MyDrive/stock/learners.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/stock
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/stock
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.2 MB/s eta 0:00:00
/content/drive/MyDrive/stock
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/stock


In [7]:
%run /content/drive/MyDrive/stock/data_manager.ipynb

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=686d10e187926c074d3d2beccd58ebce54cd00e62f8ce2ba22db65c376cc7404
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.31
    Uninstalling yfinance-0.2.31:
      Successfully uninstalled yfinance-0.2.31


In [8]:
import os
import sys
import logging
import argparse
import json

In [9]:
os.environ['VARL'] = os.path.dirname(os.path.abspath('/content/drive/MyDrive/stock'))

In [12]:
args_list = [
    '--mode', 'test',
    '--name', 'Gold',
    '--stock_code', 'GC=F',
    '--method', 'a2c',
    '--net', 'lstm',
    '--backend', 'tensorflow',
    '--start_date', '2023-11-25',
    '--end_date', '2023-12-31',
    '--lr', '0.1',
    '--discount_factor', '1.0',
    '--balance', '50000000'
]

In [15]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--mode', choices=['train', 'test', 'update', 'predict'], default='train')
    parser.add_argument('--name', default=get_time_str()+"gold")
    parser.add_argument('--stock_code', nargs='+')
    parser.add_argument('--method', default='a2c')
    parser.add_argument('--net', default='lstm')
    parser.add_argument('--backend', default='tensorflow')
    parser.add_argument('--start_date', default='2020-07-01')
    parser.add_argument('--end_date', default='2022-12-31')
    parser.add_argument('--lr', type=float, default=0.01)
    parser.add_argument('--discount_factor', type=float, default=0.9)
    parser.add_argument('--balance', type=int, default=100000000)
    parser.add_argument('--num_epoches', type=int, default=50)
    args = parser.parse_args(args_list)

    # 학습기 파라미터 설정
    output_name = f'{args.mode}_{args.name}_{args.method}_{args.net}'
    learning = args.mode in ['train', 'update']
    reuse_models = args.mode in ['test', 'update', 'predict']
    value_network_name = f'{args.name}_{args.method}_{args.net}_value.mdl'
    policy_network_name = f'{args.name}_{args.method}_{args.net}_policy.mdl'
    start_epsilon = 0.4 if args.mode in ['train', 'update'] else 0
    num_epoches = 50 if args.mode in ['train', 'update'] else 1
    num_steps = 4 if args.net in ['lstm', 'cnn'] else 1

    # Backend 설정
    os.environ['VARL'] = args.backend
    os.environ['KERAS_BACKEND'] = 'tensorflow'


    # 출력 경로 생성
    output_path = os.path.join('VARL', 'output', output_name)
    if not os.path.isdir(output_path):
        os.makedirs(output_path)

    # 파라미터 기록
    params = json.dumps(vars(args))
    with open(os.path.join(output_path, 'params.json'), 'w') as f:
        f.write(params)

    # 모델 경로 준비
    # 모델 포멧은 TensorFlow는 h5, PyTorch는 pickle
    value_network_path = os.path.join('VARL', 'models', value_network_name)
    policy_network_path = os.path.join('VARL', 'models', policy_network_name)

    # 로그 기록 설정
    log_path = os.path.join(output_path, f'{output_name}.log')
    if os.path.exists(log_path):
        os.remove(log_path)
    logging.basicConfig(format='%(message)s')
    logger = logging.getLogger(LOGGER_NAME)
    logger.setLevel(logging.DEBUG)
    logger.propagate = False
    stream_handler = logging.StreamHandler(sys.stdout)
    stream_handler.setLevel(logging.INFO)
    file_handler = logging.FileHandler(filename=log_path, encoding='utf-8')
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    logger.info(params)

    # Backend 설정, 로그 설정을 먼저하고 RLTrader 모듈들을 이후에 임포트해야 함
    #from learners
    #import ReinforcementLearner, DQNLearner, PolicyGradientLearner, ActorCriticLearner, A2CLearner, A3CLearner

    common_params = {}
    list_stock_code = []
    list_chart_data = []
    list_training_data = []
    list_min_trading_price = []
    list_max_trading_price = []

    for stock_code in args.stock_code:
        # 차트 데이터, 학습 데이터 준비
        chart_data, training_data = load_data(stock_code, args.start_date, args.end_date)

        assert len(chart_data) >= num_steps

        # 최소/최대 단일 매매 금액 설정
        min_trading_price = 2500000
        max_trading_price = 100000000

        # 공통 파라미터 설정
        common_params = {'method': args.method,
            'net': args.net, 'num_steps': num_steps, 'lr': args.lr,
            'balance': args.balance, 'num_epoches': num_epoches,
            'discount_factor': args.discount_factor, 'start_epsilon': start_epsilon,
            'output_path': output_path, 'reuse_models': reuse_models}

        # 강화학습 시작
        learner = None
        common_params.update({'stock_code': stock_code,
                'chart_data': chart_data,
                'training_data': training_data,
                'min_trading_price': min_trading_price,
                'max_trading_price': max_trading_price})
        learner = A2CLearner(**{**common_params,
                    'value_network_path': value_network_path,
                    'policy_network_path': policy_network_path})


    assert learner is not None

    if args.mode in ['train', 'test', 'update']:
        learner.run(learning=learning)
        if args.mode in ['train', 'update']:
            learner.save_models()
    elif args.mode == 'predict':
        learner.predict()


{"mode": "test", "name": "Gold", "stock_code": ["GC=F"], "method": "a2c", "net": "lstm", "backend": "tensorflow", "start_date": "2023-11-25", "end_date": "2023-12-31", "lr": 0.1, "discount_factor": 1.0, "balance": 50000000, "num_epoches": 50}
{"mode": "test", "name": "Gold", "stock_code": ["GC=F"], "method": "a2c", "net": "lstm", "backend": "tensorflow", "start_date": "2023-11-25", "end_date": "2023-12-31", "lr": 0.1, "discount_factor": 1.0, "balance": 50000000, "num_epoches": 50}
{"mode": "test", "name": "Gold", "stock_code": ["GC=F"], "method": "a2c", "net": "lstm", "backend": "tensorflow", "start_date": "2023-11-25", "end_date": "2023-12-31", "lr": 0.1, "discount_factor": 1.0, "balance": 50000000, "num_epoches": 50}
{"mode": "test", "name": "Gold", "stock_code": ["GC=F"], "method": "a2c", "net": "lstm", "backend": "tensorflow", "start_date": "2023-11-25", "end_date": "2023-12-31", "lr": 0.1, "discount_factor": 1.0, "balance": 50000000, "num_epoches": 50}
[*********************100%%*

AssertionError: ignored